In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
import popmachine, itertools, patsy, GPy, scipy

In [ ]:
machine = popmachine.Machine()

In [ ]:
# ds = machine.search(genus='pseudomonas',strain='PA01',acid='acetic', pH=[7.0,6.5], **{'mM-acid':0})
ds = machine.search(genus='pseudomonas',strain='PA01',acid='propionic', pH=[7.0,6.5], **{'mM-acid':0})
ds.log()
ds.trim(3)

In [ ]:
plt.plot(ds.data.index, ds.data.values)

In [ ]:
melt = ds.melt(norm=True)

melt['mMAcid'] = melt['mM-acid'].astype(float)

melt.head()

In [ ]:
ygp, xgp = patsy.dmatrices('od ~ scale(time) + scale(pH) + C(number) + 0', melt)

# xgp[:,2] = xgp[:,2] - xgp[:,2].min()

xgp = np.column_stack((xgp[:,-2:], np.where(xgp[:,:-2]==1)[1]))

In [ ]:
xgp

In [ ]:
plt.scatter(xgp[:,0], ygp, c=xgp[:,1])

In [ ]:
plt.plot(ds.data.values[:,ds.meta.pH==7].mean(1))
plt.plot(ds.data.values[:,ds.meta.pH!=7].mean(1))

In [ ]:
kern = GPy.kern.Hierarchical((GPy.kern.RBF(2, ARD=True, name='main'), GPy.kern.RBF(2, ARD=True, name='rep')))

gp = GPy.models.GPRegression(xgp, ygp, kern)
gp.randomize()
gp

In [ ]:
gp.optimize()
gp

In [ ]:
plt.imshow(kern.K(xgp[:50,:]))

In [ ]:
gp_null = GPy.models.GPRegression(xgp, ygp, GPy.kern.RBF(2, ARD=True))
gp_null.randomize()
gp_null

In [ ]:
gp_null.optimize()
gp_null

In [ ]:
def plotOp(mu, cov, op, xpred):
    mu = np.dot(op, mu)[:,0]
    cov = np.dot(op, np.dot(cov, op.T))

    std = np.sqrt(np.diag(cov))

    plt.plot(xpred[:50,0],mu)
    plt.fill_between(xpred[:50,0],mu-1.96*std, mu+1.96*std,alpha=.1)

In [ ]:
xpred = np.zeros((100,3))
xpred[:,0] = np.tile(np.linspace(xgp[:,0].min(), xgp[:,0].max()), 2)
xpred[:50,1] = 1
xpred[50:,1] = -1
xpred[:,2] = 100

op = np.zeros((50,100))
op[:,:50] = np.eye(50)
op[:,50:] = -np.eye(50)

mu, cov = gp.predict_noiseless(xpred, full_cov=True, kern=kern.main)

plotOp(mu, cov, op, xpred)

mu, cov = gp_null.predict_noiseless(xpred, full_cov=True)

plotOp(mu, cov, op, xpred)

plt.plot([xgp[:,0].min(), xgp[:,0].max()], [0]*2, c='k', lw=2)

In [ ]:
xpred = np.zeros((50,3))
xpred[:,0] = np.linspace(xgp[:,0].min(), xgp[:,0].max())
xpred[:,1] = 1
xpred[:,2] = 100

mu, cov = gp.predict(xpred, full_cov=True)

plt.plot(scipy.stats.multivariate_normal(mu[:,0], cov).rvs(size=10).T);